In [13]:
!pip install pydub mutagen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 1.7 MB/s eta 0:00:00a 0:00:01


In [37]:
from pydub import AudioSegment
from pydub.utils import mediainfo
import taglib
import os

def make_mono(in_file, out_file):
    sound = AudioSegment.from_wav(in_file)
    sound = sound.set_channels(1)
    tags = mediainfo(in_file).get('TAG', {})
    tags['TRACKNUMBER'] = tags['track']
    print('tags:::')
    print(tags)
    sound.export(out_file, format="wav")
    with taglib.File(out_file, save_on_exit=True) as song:
        song.tags = tags
    print(mediainfo(out_file))

for file in os.listdir('../seth/wavs'):
    make_mono("../seth/wavs/"+file, "../sethmono/wavs/"+file)

tags:::
{'title': 'Voice Synthesis 1', 'album': 'Voice Synthesis', 'artist': 'Seth Kitchen', 'date': '2023', 'track': '1', 'TRACKNUMBER': '1'}
{'index': '0', 'codec_name': 'pcm_s16le', 'codec_long_name': 'PCM signed 16-bit little-endian', 'profile': 'unknown', 'codec_type': 'audio', 'codec_tag_string': '[1][0][0][0]', 'codec_tag': '0x0001', 'sample_fmt': 's16', 'sample_rate': '22050', 'channels': '1', 'channel_layout': 'unknown', 'bits_per_sample': '16', 'id': 'N/A', 'r_frame_rate': '0/0', 'avg_frame_rate': '0/0', 'time_base': '1/22050', 'start_pts': 'N/A', 'start_time': 'N/A', 'duration_ts': '169216', 'duration': '7.674195', 'bit_rate': '354171', 'max_bit_rate': 'N/A', 'bits_per_raw_sample': 'N/A', 'nb_frames': 'N/A', 'nb_read_frames': 'N/A', 'nb_read_packets': 'N/A', 'DISPOSITION': {'default': '0', 'dub': '0', 'original': '0', 'comment': '0', 'lyrics': '0', 'karaoke': '0', 'forced': '0', 'hearing_impaired': '0', 'visual_impaired': '0', 'clean_effects': '0', 'attached_pic': '0', 'time

In [ ]:
import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.vits_config import VitsConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# we use the same path as this script as our training folder.
output_path = os.path.dirname(os.path.abspath('../../TTS'))

# DEFINE DATASET CONFIG
# Set LJSpeech as our target dataset and define its path.
# You can also use a simple Dict to define the dataset and pass it to your custom formatter.
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "SethVoice/sethmono")
)

# INITIALIZE THE TRAINING CONFIGURATION
# Configure the model. Every config class inherits the BaseTTSConfig.
config = VitsConfig(
    datasets=[dataset_config],
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = Vits(config, ap, tokenizer, speaker_manager=None)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 385 files in /root/tts-output/SethVoice/sethmono


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=output/run-July-26-2023_06+39PM-0000000

 > Model has 83046892 parameters

 > EPOCH: 0/1000
 --> output/run-July-26-2023_06+39PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 382



 > TRAINING (2023-07-26 18:39:18) 


 | > Preprocessing samples
 | > Max text length: 169
 | > Min text length: 21
 | > Avg text length: 98.1151832460733
 | 
 | > Max audio length: 222621.0
 | > Min audio length: 35994.0
 | > Avg audio length: 132076.82460732985
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
eight. The press yard.

 [!] Character '\n' not found in the vocabulary. Discarding it.


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.02729630470275879 (+0.0)
     | > avg_loss_disc: 2.4195380210876465 (+0.0)
     | > avg_loss_disc_real_0: 0.1886490285396576 (+0.0)
     | > avg_loss_disc_real_1: 0.2401406615972519 (+0.0)
     | > avg_loss_disc_real_2: 0.1882615089416504 (+0.0)
     | > avg_loss_disc_real_3: 0.23732052743434906 (+0.0)
     | > avg_loss_disc_real_4: 0.16416552662849426 (+0.0)
     | > avg_loss_disc_real_5: 0.19829007983207703 (+0.0)
     | > avg_loss_0: 2.4195380210876465 (+0.0)
     | > avg_loss_gen: 2.2000441551208496 (+0.0)
     | > avg_loss_kl: 2.2922520637512207 (+0.0)
     | > avg_loss_feat: 4.005247116088867 (+0.0)
     | > avg_loss_mel: 26.210071563720703 (+0.0)
     | > avg_loss_duration: 1.6963603496551514 (+0.0)
     | > avg_loss_1: 36.40397644042969 (+0.0)


 > EPOCH: 605/1000
 --> output/run-July-26-2023_06+39PM-0000000

 > TRAINING (2023-07-26 22:08:08) 

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.026826381683349

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.02682638168334961 (+0.0)
     | > avg_loss_disc: 2.393343687057495 (+0.0)
     | > avg_loss_disc_real_0: 0.05766531825065613 (+0.0)
     | > avg_loss_disc_real_1: 0.1973276287317276 (+0.0)
     | > avg_loss_disc_real_2: 0.22014951705932617 (+0.0)
     | > avg_loss_disc_real_3: 0.2575957775115967 (+0.0)
     | > avg_loss_disc_real_4: 0.28252655267715454 (+0.0)
     | > avg_loss_disc_real_5: 0.28741589188575745 (+0.0)
     | > avg_loss_0: 2.393343687057495 (+0.0)
     | > avg_loss_gen: 2.216707706451416 (+0.0)
     | > avg_loss_kl: 2.255918264389038 (+0.0)
     | > avg_loss_feat: 4.049737930297852 (+0.0)
     | > avg_loss_mel: 26.366771697998047 (+0.0)
     | > avg_loss_duration: 1.6727914810180664 (+0.0)
     | > avg_loss_1: 36.561927795410156 (+0.0)


 > EPOCH: 606/1000
 --> output/run-July-26-2023_06+39PM-0000000

 > TRAINING (2023-07-26 22:08:28) 

   --> TIME: 2023-07-26 22:08:33 -- STEP: 3/12 -- GLOBAL_STEP: 7275
     | > loss_dis

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.028942584991455078 (+0.0)
     | > avg_loss_disc: 2.6949515342712402 (+0.0)
     | > avg_loss_disc_real_0: 0.2772676944732666 (+0.0)
     | > avg_loss_disc_real_1: 0.2585639953613281 (+0.0)
     | > avg_loss_disc_real_2: 0.18417765200138092 (+0.0)
     | > avg_loss_disc_real_3: 0.2842058539390564 (+0.0)
     | > avg_loss_disc_real_4: 0.3106967806816101 (+0.0)
     | > avg_loss_disc_real_5: 0.33760929107666016 (+0.0)
     | > avg_loss_0: 2.6949515342712402 (+0.0)
     | > avg_loss_gen: 2.2999250888824463 (+0.0)
     | > avg_loss_kl: 2.641726016998291 (+0.0)
     | > avg_loss_feat: 3.2118172645568848 (+0.0)
     | > avg_loss_mel: 25.178159713745117 (+0.0)
     | > avg_loss_duration: 1.7232756614685059 (+0.0)
     | > avg_loss_1: 35.05490493774414 (+0.0)


 > EPOCH: 607/1000
 --> output/run-July-26-2023_06+39PM-0000000

 > TRAINING (2023-07-26 22:08:49) 
 > Keyboard interrupt detected.
 > Saving model before exiting...

 > CHECKPOINT : o

In [1]:
import os
os.path.exists('/root/tts-output/run-July-26-2023_12+16AM-0000000/config.json')

True

In [2]:
from TTS.api import Synthesizer

tts = Synthesizer(tts_checkpoint="../TTS/output/run-July-26-2023_06+39PM-0000000/best_model_3144.pth", tts_config_path="../TTS/output/run-July-26-2023_06+39PM-0000000/config.json")

#tts = TTS(model_name="/root/tts-output/run-July-26-2023_12+16AM-0000000/best_model_1908.pth", progress_bar=True, gpu=True)

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [4]:
outputs = tts.tts(text="but by printers in Paris")
tts.save_wav(outputs, "output.wav")

 > Text splitted to sentences.
['but by printers in Paris']
 > Processing time: 0.4139368534088135
 > Real-time factor: 0.20023051109302248
